In [1]:
import pandas as pd
import re
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

'''
读取训练集并构造样本
'''
def split_sentence(sentence):
    stop = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'
    sentence = re.sub(stop, '', sentence)
    return sentence.split()

data = pd.read_csv('data/labeledTrainData.tsv',sep='\t')
# data = data[:100]
sentences = data.review.apply(split_sentence)

'''
训练word2vec
'''
# 嵌入维度
embedding_vector_size = 10
w2v_model = Word2Vec(
    sentences=sentences,
    size=embedding_vector_size,
    min_count=1,
    window=3,
    workers=4,
)
# 生成词典列表
vocab_list = list(w2v_model.wv.vocab.keys())
# 生成索引
word_index = {word: index for index, word in enumerate(vocab_list)}
# 序列化
def get_index(sentence):
    global word_index
    sequence = []
    for word in sentence:
        try:
            sequence.append(word_index[word])
            # 对于每一个单词存储index
        except KeyError:
            pass
    return sequence

X_data = list(map(get_index, sentences))

max_len = 150
X_pad = pad_sequences(X_data, maxlen=max_len)
# 获取标签
Y = data.sentiment.values
# 划分数据集
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad,
    Y,
    test_size=0.2,
    random_state=42)
# random_state随机数种子

embedding_matrix = w2v_model.wv.vectors
print(embedding_matrix.shape)

(158940, 10)


In [2]:
'''
构建分类模型
'''
model = Sequential()
model.add(Embedding(
    input_dim=embedding_matrix.shape[0],
    output_dim=embedding_matrix.shape[1],
    input_length=max_len,
    weights=[embedding_matrix],
    trainable=False
))

model.add(Flatten())
model.add(Dense(5))
model.add(Dense(1,activation='sigmoid'))


model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['acc'])

history = model.fit(x=X_train,
                   y=Y_train,
                   validation_data=(X_test,Y_test),
                   batch_size=4,
                   epochs=10)

Epoch 1/10
5000/5000 [==============================] - 15s 3ms/step - loss: 0.8038 - acc: 0.5293 - val_loss: 0.6862 - val_acc: 0.5504
Epoch 2/10
5000/5000 [==============================] - 15s 3ms/step - loss: 0.6805 - acc: 0.5705 - val_loss: 0.6848 - val_acc: 0.5520
Epoch 3/10
5000/5000 [==============================] - 16s 3ms/step - loss: 0.6730 - acc: 0.5846 - val_loss: 0.6921 - val_acc: 0.5556
Epoch 4/10
5000/5000 [==============================] - 16s 3ms/step - loss: 0.6657 - acc: 0.5980 - val_loss: 0.6900 - val_acc: 0.5494
Epoch 5/10
5000/5000 [==============================] - 15s 3ms/step - loss: 0.6625 - acc: 0.6003 - val_loss: 0.6896 - val_acc: 0.5488
Epoch 6/10
5000/5000 [==============================] - 15s 3ms/step - loss: 0.6594 - acc: 0.6080 - val_loss: 0.6874 - val_acc: 0.5506
Epoch 7/10
5000/5000 [==============================] - 15s 3ms/step - loss: 0.6569 - acc: 0.6111 - val_loss: 0.7040 - val_acc: 0.5476
Epoch 8/10
5000/5000 [==============================] -